# Setup Environment

In [ ]:
# install Java8
!apt-get -q install openjdk-8-jdk-headless -qq > /dev/null

# download spark3.1.1
!wget -q https://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop2.7.tgz

# unzip it
!tar xf spark-3.1.1-bin-hadoop2.7.tgz

# install findspark 
!pip install -q findspark


import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop2.7"


import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
#spark = SparkSession.builder.appName('ops').getOrCreate()

# Load Dataset

In [ ]:
!wget -q https://raw.githubusercontent.com/muhammetsnts/SPARK/main/data/sample_linear_regression_data.txt

In [ ]:
training = spark.read.format('libsvm').load('sample_linear_regression_data.txt')

In [ ]:
training.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

# Creating and Training a LR Model

First we create a model.

In [ ]:
from pyspark.ml.regression import LinearRegression

In [ ]:
# for more parameter. Shift + Tab

lr = LinearRegression(featuresCol='features', labelCol='label', predictionCol='prediction') 

Now we can fit our data to our model.

In [ ]:
lrModel = lr.fit(training)

Lets check some statistics.

In [ ]:
lrModel.coefficients

DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [ ]:
lrModel.intercept

0.14228558260358093

Lets check the summary attribute that contains even mre information.

In [ ]:
training_summary = lrModel.summary

In [ ]:
training_summary.r2  # how much variance is explained by the model

0.027839179518600154

In [ ]:
training_summary.rootMeanSquaredError

10.16309157133015

In [ ]:
training_summary.pValues

[0.9927505031240562,
 0.30967074330990396,
 0.3178269194409711,
 0.003972477331573909,
 0.5201486327242175,
 0.16213017210149872,
 0.7102819001865635,
 0.5266812209137877,
 0.46256007153356316,
 0.37825808848978526,
 0.7592692146070568]

# Train-Test Split

In [ ]:
all_data = spark.read.format('libsvm').load('sample_linear_regression_data.txt')

In [ ]:
# there are two kind of dataframe 
# first one is 0.7 of the data and the second one is 0.3

split_object = all_data.randomSplit([0.7, 0.3])
split_object

[DataFrame[label: double, features: vector],
 DataFrame[label: double, features: vector]]

But we generally refer this one.

In [ ]:
train_data, test_data = all_data.randomSplit([0.7, 0.3])

In [ ]:
train_data.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
|-28.046018037776633|(10,[0,1,2,3,4,5,...|
|-26.736207182601724|(10,[0,1,2,3,4,5,...|
| -23.51088409032297|(10,[0,1,2,3,4,5,...|
|-23.487440120936512|(10,[0,1,2,3,4,5,...|
|-22.949825936196074|(10,[0,1,2,3,4,5,...|
|-22.837460416919342|(10,[0,1,2,3,4,5,...|
|-21.432387764165806|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-19.872991038068406|(10,[0,1,2,3,4,5,...|
| -19.66731861537172|(10,[0,1,2,3,4,5,...|
|-19.402336030214553|(10,[0,1,2,3,4,5,...|
|-18.845922472898582|(10,[0,1,2,3,4,5,...|
|-17.494200356883344|(10,[0,1,2,3,4,5,...|
|-17.428674570939506|(10,[0,1,2,3,4,5,...|
| -17.32672073267595|(10,[0,1,2,3,4,5,...|
|-17.065399625876015|(10,[0,1,2,3,4,5,...|
|-17.026492264209548|(10,[0,1,2,3,4,5,...|
| -16.26143027545273|(10,[0,1,2,3,4,5,...|
|-16.151349351277112|(10,[0,1,2,3,4,5,...|
| -16.08565904102149|(10,[0,1,2,3,4,5,...|
+----------

In [ ]:
test_data.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
|-28.571478869743427|(10,[0,1,2,3,4,5,...|
|-26.805483428483072|(10,[0,1,2,3,4,5,...|
|-20.212077258958672|(10,[0,1,2,3,4,5,...|
|-19.884560774273424|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
| -18.27521356600463|(10,[0,1,2,3,4,5,...|
|-17.803626188664516|(10,[0,1,2,3,4,5,...|
| -16.71909683360509|(10,[0,1,2,3,4,5,...|
|-16.692207021311106|(10,[0,1,2,3,4,5,...|
| -15.86200932757056|(10,[0,1,2,3,4,5,...|
|-15.732088272239245|(10,[0,1,2,3,4,5,...|
| -15.72351561304857|(10,[0,1,2,3,4,5,...|
|-15.334767479922341|(10,[0,1,2,3,4,5,...|
|-15.310980589416289|(10,[0,1,2,3,4,5,...|
|-15.056482974542433|(10,[0,1,2,3,4,5,...|
|-14.762758252931127|(10,[0,1,2,3,4,5,...|
|-13.976130931152703|(10,[0,1,2,3,4,5,...|
|-13.867087895158768|(10,[0,1,2,3,4,5,...|
| -13.15333560636553|(10,[0,1,2,3,4,5,...|
+----------

In [ ]:
test_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                148|
|   mean|-0.9088682857352514|
| stddev| 10.342590885962913|
|    min|-28.571478869743427|
|    max| 24.290551295953957|
+-------+-------------------+



# Evaluate Model

In [ ]:
correct_model = lr.fit(train_data)

In [ ]:
test_results = correct_model.evaluate(test_data)

In [ ]:
test_results.rootMeanSquaredError

10.70132357350396

In [ ]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
|-28.949955352611447|
|  -29.9599462749756|
|-22.338769093200554|
|-21.247229601601216|
|-21.340785497565232|
|  -17.4733219348941|
|-18.531849277768732|
|-15.235795799393944|
|-17.309094908162105|
| -17.11448518904557|
| -18.75048204406387|
|-15.055586480022734|
|-20.539072887323467|
| -16.49709477556081|
|-12.350619305341677|
|-18.579267763348717|
| -16.35578187170862|
|-14.223857979504098|
|-11.049621880112518|
|-12.552441642925524|
+-------------------+
only showing top 20 rows



# Deploying Model

Lets create a unlabeled data from test data.

In [ ]:
unlabeled_data = test_data.select('features')
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows



Lets transform our unlabeled data and get predictions.

In [ ]:
predictions = correct_model.transform(unlabeled_data)

In [ ]:
predictions.show()

+--------------------+-------------------+
|            features|         prediction|
+--------------------+-------------------+
|(10,[0,1,2,3,4,5,...|0.37847648286801994|
|(10,[0,1,2,3,4,5,...| 3.1544628464925264|
|(10,[0,1,2,3,4,5,...|  2.126691834241883|
|(10,[0,1,2,3,4,5,...| 1.3626688273277918|
|(10,[0,1,2,3,4,5,...| 1.5580227079506948|
|(10,[0,1,2,3,4,5,...|-1.6949706880696627|
|(10,[0,1,2,3,4,5,...| 0.2566357117640999|
|(10,[0,1,2,3,4,5,...|-2.5678303892705725|
|(10,[0,1,2,3,4,5,...| 0.5899980745570164|
|(10,[0,1,2,3,4,5,...|0.42227816773446536|
|(10,[0,1,2,3,4,5,...| 2.8884727164933084|
|(10,[0,1,2,3,4,5,...|-0.6765017922165105|
|(10,[0,1,2,3,4,5,...|  4.815557274274899|
|(10,[0,1,2,3,4,5,...|  1.162327295638471|
|(10,[0,1,2,3,4,5,...| -2.960361284074611|
|(10,[0,1,2,3,4,5,...| 3.5227847888062858|
|(10,[0,1,2,3,4,5,...|  1.593023618777492|
|(10,[0,1,2,3,4,5,...|0.24772704835139592|
|(10,[0,1,2,3,4,5,...| -2.817466015046249|
|(10,[0,1,2,3,4,5,...|-0.6008939634400053|
+----------